# Forecasting Pipeline

In [1]:
## add in overview

# Prerequisites 

This example runs on an Azure Machine Learning Notebook VM. We are calling models that have **already been trained and registered** to the Workspace. If you have already run the Environment Setup and Training Pipeline notebooks or you have an AML Notebook set up with Models registered to the Workspace you are all set. 

## Setup the Workspace, Datastore, and Compute

As we did in the Training Pipeline notebook, we need to call the Workspace. We also want to create variables for the datastore and compute cluster. 

### Connect to the workspace
Create a workspace object. Workspace.from_config() reads the file config.json and loads the details into an object named ws. 

In [ ]:
from azureml.core import Workspace 

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

### Attach existing compute resource
From the Environment Setup Notebook, we created a compute cluster. 

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

compute = AmlCompute(ws, 'many-models')

### Call the Datastore containing the Orange Juice sales data
From the Generate Data Notebook, we uploaded the csv's for each Store and Brand comination. Use the .get_default_datastore() to save the datastore we uploaded the files into. 

In [ ]:
dstore = ws.get_default_datastore()

### Call Registerd FileDataset
In the Data_Prep notebook, we uploaded our data to Blob storage then registered the folder of data as a FileDataset to the Workspace. We are call that Dataset in order to pass it as an input into our ParallelRunStep. 

In [ ]:
# This cell reads in subset and registered data from the dataprep notebook 
from azureml.pipeline.core import Pipeline, PipelineData

ds_name = '10modelsfiledataset'
oj_ds = Dataset.get_by_name(ws, name = ds_name)

### Subset the number of models to run
If you would like to test the scoring pipeline with a subset of models use the following code to take a sample of the full dataset. To make sure the same subset is selected across the Training, Scoring and Forecasting Notebooks use the same seed value. Otherwise, run the commented out cell to run all 11,000+ models. 

In [ ]:
oj_ds_subset = oj_ds.take_sample(0.01, seed = 1248)

oj_input_data = oj_ds_subset.as_named_input('oj_series')

# This line will create a dataset for the full 11,973 models. 
#oj_input_data = oj_ds.as_named_input('oj_series')

## Create entry script for Forecasting
To use the models to make forecasts, you need an **entry script** and a list of **dependencies**:

#### An entry script
This script accepts requests, scores the requests by using the model, and returns the results.
- __init()__ - Typically this function loads the model into a global object. This function is run only once at the start of batch processing per worker node/process. init method can make use of following environment variables (ParallelRunStep input):
    1.	AZUREML_BI_OUTPUT_PATH – output folder path
- __run(mini_batch)__ - The method to be parallelized. Each invocation will have one minibatch.<BR>
__mini_batch__: Batch inference will invoke run method and pass either a list or Pandas DataFrame as an argument to the method. Each entry in mini_batch will be - a filepath if input is a FileDataset, a Pandas DataFrame if input is a TabularDataset.<BR>
__run__ method response: run() method should return a Pandas DataFrame or an array. For append_row output_action, these returned elements are appended into the common output file. For summary_only, the contents of the elements are ignored. For all output actions, each returned output element indicates one successful inference of input element in the input mini-batch.
    User should make sure that enough data is included in inference result to map input to inference. Inference output will be written in output file and not guaranteed to be in order, user should use some key in the output to map it to input.
    

#### Dependencies
Helper scripts or Python/Conda packages required to run the entry script or model.

The deployment configuration for the compute target that hosts the deployed model. This configuration describes things like memory and CPU requirements needed to run the model.

These items are encapsulated into an inference configuration and a deployment configuration. The inference configuration references the entry script and other dependencies. You define these configurations programmatically when you use the SDK to perform the deployment. You define them in JSON files when you use the CLI.


## Build and Run the batch inferece pipeline
Now that the data, models, and compute resources are set up, we can put together a pipeline for forecasting. 
### Set up the environment to run the script
Specify the conda dependencies for your script. This will allow us to install packages and configure the environment. 

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

# set up the batch environment settings
batch_conda_deps = CondaDependencies.create(pip_packages=['sklearn','pmdarima'])

batch_env = Environment(name="manymodels_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE

### Create the configuration to wrap the inference script 
In the [ParallelRunConfig](https://docs.microsoft.com/en-us/python/api/azureml-contrib-pipeline-steps/azureml.contrib.pipeline.steps.parallelrunconfig?view=azure-ml-py), you will want to determine the number of workers and nodes appropriate for your use case. 
The _workercount_ is based off the number of cores on the VM. 
The _nodecount_ will determine the number of nodes to use. Increasing the node count should help to speed up the process. 

**include info here about runs we tried**

You should set the _timeout_ to be the slightly longer than amount of time it would take for one iteration of your script to complete. In this example, that would be the amount of time to pull down a model and make predictions. The default time is 60 seconds. 

We added tags for additional information about our settings for the step. 

In [ ]:
from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig 

workercount = 5
nodecount = 8
timeout = 3000

compute = AmlCompute(ws, "train-max")

tags1 = {}
tags1['nodes'] = nodecount
tags1['workers-per-node'] = workercount
tags1['timeout'] = timeout 

parallel_run_config = ParallelRunConfig(
    source_directory = './scripts',
    entry_script = 'score.py',
    mini_batch_size = '1',
    run_invocation_timeout = timeout, 
    error_threshold = 10,
    output_action = 'summary_only', 
    environment = batch_env, 
    process_count_per_node = workercount, 
    compute_target = compute, 
    node_count = nodecount
)

### Create the ParallelRunStep
This is where we will call the entry script, environment configuration, and parameters. This [ParallelRunStep](https://docs.microsoft.com/en-us/python/api/azureml-contrib-pipeline-steps/azureml.contrib.pipeline.steps.parallel_run_step.parallelrunstep?view=azure-ml-py) is the main step in our pipeline.  

#### We can specify the following parameters: 
- _input_ : We will provide the data that will be used in the entry_script. 
- _output_ : This is the directory where the output of the step will be written. 
- _models_ : This provides additional metadata about the models used in the step. 
- _arguments_ : You can specify arguments that you want to pass to the entry_script with this argument.

_arguments_ and _inputs_ are the two parameters that can pass information to the entry_script. 

For the orange juice sales forecasting, we have two arguments passed to the entry_script. *n_test_set* is the length of the test set which is also the number of predictions you would like to make. *timestamp_column* is the date column from the data. 

In [ ]:
datasetname = 'store'
output_dir = PipelineData(name = 'forecasting_output', 
                         datastore = dstore, 
                         output_path_on_compute = 'forecasting_output/')

parallelrun_step = ParallelRunStep(
    name="many-models-scoring",
    parallel_run_config=parallel_run_config,
    inputs=[oj_input_data],  
    output=output_dir,
    models= [], # this is just for logging
    arguments=['--forecast_horizon', 8,
              '--starting_date', '1992-10-01'],
    allow_reuse = False
)

## Forecasting Script

In [ ]:
%%writefile ./scripts/forecasting.py

import pandas as pd
import os
import uuid
import argparse
import datetime
import numpy as np
from sklearn.externals import joblib
from joblib import dump, load
import pmdarima as pm
import time
from datetime import timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error 
import pickle
import logging 

# Import the AzureML packages 
from azureml.core.model import Model
from azureml.core import Experiment, Workspace, Run
from azureml.core import ScriptRunConfig
from azureml.core.run import Run

# Import the helper script 
from entry_script_helper import EntryScriptHelper


# Get the information for the current Run
thisrun = Run.get_context()

# Set the log file name
LOG_NAME = "user_log"

# Parse the arguments passed in the PipelineStep through the arguments option 
parser = argparse.ArgumentParser("split")
parser.add_argument("--forecast_horizon", type=int, help="input number of predictions")
parser.add_argument("--starting_date", type=str, help="date to begin forcasting")

args, unknown = parser.parse_known_args()

print("Argument 1(forecast_horizon): %s" % args.forecast_horizon)
print("Argument 2(starting_date): %s" % args.starting_date)


def init():
    EntryScriptHelper().config(LOG_NAME)
    logger = logging.getLogger(LOG_NAME)
    output_folder = os.path.join(os.environ.get("AZ_BATCHAI_INPUT_AZUREML", ""), "temp/output")
    logger.info(f"{__file__}.output_folder:{output_folder}")
    logger.info("init()")    
    return

def run(input_data):
    print("begin run ")
    
    # 0. Set up Logging
    logger = logging.getLogger(LOG_NAME)
    os.makedirs('./outputs', exist_ok=True)
    resultsList = []
    #predictions = pd.DataFrame()
    logger.info('making forecasts...')
    
    print('looping through data')
    # 1. Loop through the input data 
    for idx, file in enumerate(input_data): # add the enumerate for the 12,000 files 
        u1 = uuid.uuid4()
        mname='arima'+str(u1)[0:16]        
        logs = []

        date1=datetime.datetime.now()
        logger.info('starting ('+file+') ' + str(date1))
        thisrun.log(mname,'starttime-'+str(date1))

        # 2. Set up data to predict on 
        store = [str(file).split('/')[-1][:-4].split('_')[0]] * args.forecast_horizon
        brand = [split('/')[-1][:-4].split('_')[-1]] * args.forecast_horizon
        date_list = pd.date_range(args.starting_date, periods = args.forecast_horizon, freq ='W-THU')
        
        prediciton_df = pd.DataFrame(list(zip(date_list, store, brand)), 
                                    columns = ['WeekStarting', 'Store', 'Brand'])
        
        # 3. Unpickle Model and Make Predictions             
        model_name = 'arima_'+str(file).split('/')[-1][:-4]  
        model_path = Model.get_model_path(model_name)         
        model = joblib.load(model_path)        
        
        prediction_list, conf_int = model.predict(args.forecast_horizon, return_conf_int = True)

        prediction_df['Predictions'] = prediction_list

        # 4. Save the output back to blob storage 
        run_date = datetime.datetime.now().date()
        ws1 = thisrun.experiment.workspace
        output_path = os.path.join('./outputs/', model_name + str(run_date))
        test.to_csv(path_or_buf=output_path + '.csv', index = False)
        dstore = ws1.get_default_datastore()
        dstore.upload_files([output_path + '.csv'], target_path='oj_forecasts' + str(run_date), overwrite=False, show_progress=True)

        # 5. Append the predictions to return a dataframe if desired 

        # 6. Log Metrics
        date2=datetime.datetime.now()
        logger.info('ending ('+str(file)+') ' + str(date2))

        logs.append(str(file).split('/')[-1][:-4])
        logs.append(model_name)
        logs.append(str(date1))
        logs.append(str(date2))
        logs.append(date2-date1)
        logs.append(idx)
        logs.append(len(input_data))
        logs.append(thisrun.get_status())        

        thisrun.log(mname,'endtime-'+str(date2))
        thisrun.log(mname,'auc-1')

    resultsList.append(logs)
    return resultsList